In [1]:
import pandas as pd
import enchant   
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas import DataFrame
from openpyxl import Workbook
import xlsxwriter
import math
from collections import Counter
import os

In [4]:
def read_excel(fileName, rowCount, worksheet, ML, successListWorksheet_3, nameCount_3, failuresListWorksheet_3, successListWorksheet_2, failuresListWorksheet_2, nameCountF_3, nameCount_2, nameCountF_2):
    AL = pd.read_excel(fileName)
    columns_list = AL.columns

    AL.rename(columns = {columns_list[0]:'Attendee', 
                        'Unnamed: 1':'Score','Unnamed: 2':'    1:46 PM','Unnamed: 3':'        1:55 PM','Unnamed: 4':'     2:01 PM',
                        'Unnamed: 5':'       2:41 PM'},inplace=True)
    AL= AL.drop(AL.index [  0:4  ] )
    AL= AL.drop(AL.index [  0:2  ] )
    AL.drop(AL.tail(1).index,inplace=True)
    print(AL)
    master_count=0
    for i in ML.iterrows():
        master_count=master_count+1;
    print(master_count)

    Attendance_count=0
    directmatch=0
#     cosineL=[]
#     cosinematch=0
    n_gram_list=[]
    count_n_gram_match=0
    n_gram_success_3=[]
    n_gram_failure_3=[]
    n_gram_success_count_3=0
    n_gram_failure_count_3=0
    n_gram_success_2=[]
    n_gram_failure_2=[]
    n_gram_success_count_2=0
    n_gram_failure_count_2=0
    
    
    bas_name = os.path.basename(fileName)
    split_name = bas_name.split(".")[0]
    successListWorksheet_3.write(nameCount_3, 0, split_name)
    failuresListWorksheet_3.write(nameCountF_3, 0, split_name)
    nameCount_3 = nameCount_3+1
    nameCountF_3 = nameCountF_3+1
    successListWorksheet_2.write(nameCount_2, 0, split_name)
    failuresListWorksheet_2.write(nameCountF_2, 0, split_name)
    nameCount_2 = nameCount_2+1
    nameCountF_2 = nameCountF_2+1
    countF_3 = 0
    countF_2 = 0
    for i, row in AL.iterrows():
        Attendance_count+=1
        Attendeename=f"{row['Attendee']}"
        upper_case_AN=Attendeename.upper()
        sorted_Attendeename= ''.join(sorted(upper_case_AN))
        ans=900
        flag=0
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Name']}"
            Scholar_number = f"{row2['Redg_Number']}"
            if(Mastername==upper_case_AN):
                flag=1
                break
            
        if(flag==1):
            directmatch=directmatch+1
        else:
            n_gram_list.append(Attendeename)
            count_n_gram_match+=1
    #print(cosineL, attencount, directmatch )
    
    
    def ngram(sentence, num):
        tmp = [] 
        sent_len = len(sentence) - num +1
        for i in range(sent_len):
            tmp.append(sentence[i:i+num]) 
        return tmp


    def diff_ngram(sent_a, sent_b, num):
        a = ngram(sent_a, num)
        b = ngram(sent_b, num) 
        common = [] 
        cnt = 0 
        for i in a:
            for j in b:
                if i == j:
                    cnt += 1
                    common.append(i)
                    break
        return cnt/len(a), common




    for i in n_gram_list:
        attendee_name=i
        ans=-1
        req_list=[]
        upper_case_attendee_name=attendee_name.upper()
        upper_case_removed=upper_case_attendee_name.replace(" ", "")
        sorted_attendee_name= ''.join(sorted(upper_case_removed))
        for j, row2 in ML.iterrows():
            Master_name=f"{row2['Name']}"
            Scholar_number = f"{row2['Redg_Number']}"
            master_name_removed=Master_name.replace(" ", "")
            sorted_master_name= ''.join(sorted(master_name_removed))
            val,val_list=diff_ngram(str(sorted_attendee_name), str(sorted_master_name),3)
            if(val>=ans):
                ans=val
                resultant_mastername= Master_name
                resultant_scholarnumber= Scholar_number
        print(attendee_name+"   "+resultant_mastername+"  "+str(ans))
        
        if(ans==1.0):
            n_gram_success_count_3+=1
            successListWorksheet_3.write(nameCount_3, 0, resultant_mastername)
            successListWorksheet_3.write(nameCount_3, 1, resultant_scholarnumber)
            nameCount_3 = nameCount_3+1  
        else:
            n_gram_failure_count_3+=1
            failuresListWorksheet_3.write(nameCountF_3, 0, attendee_name)
            countF_3 = countF_3+1
            nameCountF_3 = nameCountF_3+1
            
            
    for i in n_gram_list:
        attendee_name=i
        ans=-1
        req_list=[]
        upper_case_attendee_name=attendee_name.upper()
        upper_case_removed=upper_case_attendee_name.replace(" ", "")
        sorted_attendee_name= ''.join(sorted(upper_case_removed))
        for j, row2 in ML.iterrows():
            Master_name=f"{row2['Name']}"
            Scholar_number = f"{row2['Redg_Number']}"
            master_name_removed=Master_name.replace(" ", "")
            sorted_master_name= ''.join(sorted(master_name_removed))
            val,val_list=diff_ngram(str(sorted_attendee_name), str(sorted_master_name),2)
            if(val>=ans):
                ans=val
                resultant_mastername= Master_name
                resultant_scholarnumber= Scholar_number
        print(attendee_name+"   "+resultant_mastername+"  "+str(ans))
        
        if(ans==1.0):
            n_gram_success_count_2+=1
            successListWorksheet_2.write(nameCount_2, 0, resultant_mastername)
            successListWorksheet_2.write(nameCount_2, 1, resultant_scholarnumber)
            nameCount_2 = nameCount_2+1
        else:
            n_gram_failure_count_2+=1
            failuresListWorksheet_2.write(nameCountF_2, 0, attendee_name)
            countF_2 = countF_2+1
            nameCountF_2 = nameCountF_2+1
            
    if countF_3 == 0:
        failuresListWorksheet_3.write(nameCountF_3, 0, "No Failures")
        nameCountF_3 = nameCountF_3+1
        
    if countF_2 == 0:
        failuresListWorksheet_2.write(nameCountF_2, 0, "No Failures")
        nameCountF_2 = nameCountF_2+1    
        
    #print(cosineS1,cosineS2,cosineF)
    nameCount_3 = nameCount_3+1 
    nameCountF_3 = nameCountF_3+1
    nameCount_2 = nameCount_2+1 
    nameCountF_2 = nameCountF_2+1
            
     
    master_count
    x=Attendance_count/master_count
    y=x*100
    total_per=round(y,2)
    directmatch
    total_per
    per_dir_str_match=directmatch/Attendance_count 
    per_dir_str_match=per_dir_str_match*100
    per_dir_str_match=round(per_dir_str_match,2)
    n_gram_str_match=(count_n_gram_match)/Attendance_count
    n_gram_str_match=n_gram_str_match*100
    n_gram_str_match=round(n_gram_str_match,2)
    n_gram_success_percentage_3=n_gram_success_count_3/Attendance_count
    n_gram_success_percentage_3=n_gram_success_percentage_3*100
    n_gram_success_percentage_3=round(n_gram_success_percentage_3,2)
    n_gram_success_percentage_3
    n_gram_failure_percentage_3=n_gram_failure_count_3/Attendance_count
    n_gram_failure_percentage_3=n_gram_failure_percentage_3*100
    n_gram_failure_percentage_3=round(n_gram_failure_percentage_3,2)
    n_gram_failure_percentage_3
    n_gram_success_percentage_2=n_gram_success_count_2/Attendance_count
    n_gram_success_percentage_2=n_gram_success_percentage_2*100
    n_gram_success_percentage_2=round(n_gram_success_percentage_2,2)
    n_gram_success_percentage_2
    n_gram_failure_percentage_2=n_gram_failure_count_2/Attendance_count
    n_gram_failure_percentage_2=n_gram_failure_percentage_2*100
    n_gram_failure_percentage_2=round(n_gram_failure_percentage_2,2)
    n_gram_failure_percentage_2
    
    worksheet.write(0, 0,'Date')
    worksheet.write(0, 1, 'Total percentage of attendee')
    worksheet.write(0, 2, 'Total percentage of direct string match' )
    worksheet.write(0, 3, 'Total percentage of n-gram match' )
    worksheet.write(0, 4, 'percentage of 3-gram match with value as 1.0' )
    worksheet.write(0, 5, 'Failure percentage with 3-gram(Value < 1.0)' )
    worksheet.write(0, 6, 'percentage of 2-gram match with value as 1.0' )
    worksheet.write(0, 7, 'Failure percentage with 2-gram(Value < 1.0)' )
    worksheet.write(0, 8, 'Number of Attendee')
    worksheet.write(0, 9, 'Number of direct string match')
    worksheet.write(0, 10, 'Number of n-gram string match')
    worksheet.write(0, 11, 'Number of success with 3-gram (Value==1.0)')
    worksheet.write(0, 12, 'Number of failure with 3-gram( Value<1.0)')
    worksheet.write(0, 13, 'Number of success with 2-gram (Value==1.0)')
    worksheet.write(0, 14, 'Number of failure with 2-gram( Value<1.0)')
    
    date_file = fileName.split(".")
    date_tbu = date_file[0]

    worksheet.write(rowCount,0,date_tbu)
    worksheet.write(rowCount, 1, total_per)
    worksheet.write(rowCount, 2, per_dir_str_match )
    worksheet.write(rowCount, 3, n_gram_str_match)
    worksheet.write(rowCount, 4, n_gram_success_percentage_3)
    worksheet.write(rowCount, 5, n_gram_failure_percentage_3)
    worksheet.write(rowCount, 6, n_gram_success_percentage_2)
    worksheet.write(rowCount, 7, n_gram_failure_percentage_2)
    worksheet.write(rowCount, 8, Attendance_count )
    worksheet.write(rowCount, 9, directmatch)
    worksheet.write(rowCount, 10, count_n_gram_match)
    worksheet.write(rowCount, 11, n_gram_success_count_3)
    worksheet.write(rowCount, 12, n_gram_failure_count_3)
    worksheet.write(rowCount, 13, n_gram_success_count_2)
    worksheet.write(rowCount, 14, n_gram_failure_count_2)
    return (nameCount_3, nameCountF_3,nameCount_2, nameCountF_2)

def checkIfXlsx(fileName):
    fileData = fileName.split(".")
    fileExt = fileData[-1]
    fileBaseName = os.path.basename(fileName)
    if(fileBaseName=='n-gram_3_Set.xlsx' or fileBaseName=='MasterStudentList-3.xlsx' or
        fileBaseName=='Output_Success_3gram.xlsx' or fileBaseName=='Output_Failures_3gram.xlsx'  or
        fileBaseName=='Output_Success_2gram.xlsx' or fileBaseName=='Output_Failures_2gram.xlsx'):
        return False
    if(fileExt=='xlsx'):
        return True
    return False

if __name__ == '__main__':
    ML = pd.read_excel("MasterStudentList-3.xlsx")
    workbook = xlsxwriter.Workbook("n-gram_3_Set.xlsx",)
    successListWorkbook_3 = xlsxwriter.Workbook("Output_Success_3gram.xlsx")
    failuresListWorkbook_3 = xlsxwriter.Workbook("Output_Failures_3gram.xlsx")
    successListWorkbook_2 = xlsxwriter.Workbook("Output_Success_2gram.xlsx")
    failuresListWorkbook_2 = xlsxwriter.Workbook("Output_Failures_2gram.xlsx")
    worksheet = workbook.add_worksheet()
    successListWorksheet_3 = successListWorkbook_3.add_worksheet()
    failuresListWorksheet_3 = failuresListWorkbook_3.add_worksheet()
    successListWorksheet_2 = successListWorkbook_2.add_worksheet()
    failuresListWorksheet_2 = failuresListWorkbook_2.add_worksheet()
    dir_list = os.listdir()
    fileCount = 0
    nameCount_3 = 0
    nameCountF_3 = 0
    nameCount_2 = 0
    nameCountF_2 = 0
    for file in dir_list:
        if(checkIfXlsx(fileName=file)):
            fileCount = fileCount+1
            (camCount_3, camCount_f_3, camCount_2, camCount_f_2 ) = read_excel(file, fileCount, worksheet, ML, successListWorksheet_3, nameCount_3, failuresListWorksheet_3, successListWorksheet_2, failuresListWorksheet_2, nameCountF_3, nameCount_2 ,nameCountF_2)
            nameCount_3 = camCount_3
            nameCountF_3 = camCount_f_3
            nameCount_2 = camCount_2
            nameCountF_2 = camCount_f_2
            
            
    workbook.close()
    successListWorkbook_3.close()
    failuresListWorkbook_3.close()
    successListWorkbook_2.close()
    failuresListWorkbook_2.close()

                 Attendee Score     1:46 PM         1:55 PM
6              AMIT KUMAR   2/2           ✅               ✅
7   ANUMANCHIPALLI ADITYA   2/2           ✅               ✅
8            ANURAG KUMAR   2/2           ✅               ✅
9    CHENNAKESAVULA KUMAR   2/2           ✅               ✅
10           Gaurav KUMAR   2/2           ✅               ✅
11           GONDU GANESH   2/2           ✅               ✅
12         KARUSODU KUMAR   2/2           ✅               ✅
13       KOTTAPALLI KUMAR   2/2           ✅               ✅
14         MD IMRAN ANJUM   2/2           ✅               ✅
15           MITHUN KUMAR   2/2           ✅               ✅
16          NIRAKAR BHOLO   2/2           ✅               ✅
17        Niteshkumar Sah   2/2           ✅               ✅
18     PANKAJKUMAR SHARMA   2/2           ✅               ✅
19        POTRAKANTI SYAM   2/2           ✅               ✅
20         PRADHAN GANESH   2/2           ✅               ✅
21              RAJ ARYAN   2/2         

Abhimanyu Kumar   ABHIMANYU  KUMAR  1.0
ANURAG KUMAR   ANURAG  KUMAR  1.0
JYOTI KUMARI   JYOTI  KUMARI  1.0
MD IMRAN ANJUM   MDIMRAN  ANJUM  1.0
NIDHI KUMARI   NIDHI  KUMARI  1.0
NIRAKAR BHOLO   NIRAKAR  BHOLO  1.0
PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  1.0
PRADHAN GANESH   PRADHAN  GANESH  1.0
RAJ ARYAN   RAJ  ARYAN  1.0
RAJ GAUTAM   RAJ  GAUTAM  1.0
RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  1.0
Shashikant Raj   SHASHIKANT  RAJ  1.0
AMIT KUMAR   AMIT  KUMAR  1.0
Niteshkumar Sah   NITESH KUMAR  SAH  1.0
POTRAKANTI SYAM   POTRAKANTI  SYAM  1.0
SANDEEP KUMAR   SANDEEP  KUMAR  1.0
SATYAM SIDDHARTH   SATYAM  SIDDHARTH  1.0
Abhimanyu Kumar   ABHIMANYU  KUMAR  1.0
ANURAG KUMAR   ANURAG  KUMAR  1.0
JYOTI KUMARI   JYOTI  KUMARI  1.0
MD IMRAN ANJUM   MDIMRAN  ANJUM  1.0
NIDHI KUMARI   NIDHI  KUMARI  1.0
NIRAKAR BHOLO   NIRAKAR  BHOLO  1.0
PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  1.0
PRADHAN GANESH   PRADHAN  GANESH  1.0
RAJ ARYAN   RAJ  ARYAN  1.0
RAJ GAUTAM   RAJ  GAUTAM  1.0
RITESH KU

SAKSHI KUMARI   SAKSHI  KUMARI  1.0
SANDEEP KUMAR   SANDEEP  KUMAR  1.0
SATISHKUMAR SINGH   SATISH KUMAR  SINGH  1.0
VYSYARAJU NIKHI   VYSYARAJU SRI NIKHI  1.0
VYSYARAJU TARUN   VYSYARAJU  TARUN  1.0
ANUMANCHIPALLI ADITYA   ANUMANCHIPALLI VENKATA ADITYA  0.8421052631578947
BABLU KUMAR   BABLU  KUMAR  1.0
CHENNAKESAVULA KUMAR   CHENNAKESAVULA SUMANTH KUMAR  0.9444444444444444
CHIRANJEE V   CHIRANJEEV    1.0
GANGULA TARUN   GANGULA  TARUN  1.0
Gaurav KUMAR   GAURAV  KUMAR  1.0
GONDU GANESH   GONDU  GANESH  1.0
JYOTI KUMARI   JYOTI  KUMARI  1.0
Kanhaiya KANHAIYA JEE   KANHAIYA  JEE  0.7222222222222222
KISHAN KUMAR   KISHAN  KUMAR  1.0
KOTRA NIHARIKA   KOTRA  NIHARIKA  1.0
MANISH KUMAR   MANISH  KUMAR  1.0
MD IMRAN ANJUM   MDIMRAN  ANJUM  1.0
NAV RATNA BADAL   NAVRATNA  BADAL  1.0
NIDHI KUMARI   NIDHI  KUMARI  1.0
Niteshkumar Sah   NITESH KUMAR  SAH  1.0
PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  1.0
PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  1.0
POTRAKANTI SYAM   POTRAKANTI  SYAM  1.0
RAJ ARYA

JYOTI KUMARI   JYOTI  KUMARI  1.0
NIRAKAR BHOLO   NIRAKAR  BHOLO  1.0
PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  1.0
RAJ GAUTAM   RAJ  GAUTAM  1.0
RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  1.0
SANDEEP KUMAR   SANDEEP  KUMAR  1.0
SATISHKUMAR SINGH   SATISH KUMAR  SINGH  1.0
SATYAM SIDDHARTH   SATYAM  SIDDHARTH  1.0
Shashikant Raj   SHASHIKANT  RAJ  1.0
Kanhaiya KANHAIYA JEE   KANHAIYA  JEE  0.7222222222222222
MITHUN KUMAR   MITHUN  KUMAR  1.0
Niteshkumar Sah   NITESH KUMAR  SAH  1.0
POTRAKANTI SYAM   POTRAKANTI  SYAM  1.0
Gaurav KUMAR   GAURAV  KUMAR  1.0
NAV RATNA BADAL   NAVRATNA  BADAL  1.0
                 Attendee Score     1:46 PM         1:55 PM      2:01 PM
6         Abhimanyu Kumar   3/3           ✅               ✅            ✅
7              AMIT KUMAR   3/3           ✅               ✅            ✅
8            GONDU GANESH   3/3           ✅               ✅            ✅
9          KARUSODU KUMAR   3/3           ✅               ✅            ✅
10           NIDHI KUMARI   3/3  

MD SHAHID AKIL   MD SHAHID  AKIL  1.0
NIDHI KUMARI   NIDHI  KUMARI  1.0
Niteshkumar Sah   NITESH KUMAR  SAH  1.0
P SRILAYA   P  SRILAYA  1.0
PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  1.0
PRADHAN GANESH   PRADHAN  GANESH  1.0
RAHUL KUMAR   RAHUL  KUMAR  1.0
RAJ ARYAN   RAJ  ARYAN  1.0
SATISHKUMAR SINGH   SATISH KUMAR  SINGH  1.0
UTTAM KUMAR   UTTAM  KUMAR  1.0
VEMPATI SHANKAR   VEMPATI JAYA SHANKAR  0.8333333333333334
VYSYARAJU NIKHI   VYSYARAJU SRI NIKHI  0.8333333333333334
VYSYARAJU TARUN   VYSYARAJU  TARUN  1.0
BINODINI KAR   BINODINI  KAR  1.0
GANGULA TARUN   GANGULA  TARUN  1.0
JYOTI KUMARI   JYOTI  KUMARI  1.0
KARUSODU KUMAR   KARUSODU  LAVANKUMAR  0.7272727272727273
VOONA NIDHI   VOONA SRI NIDHI  0.875
Abhimanyu Kumar   ABHIMANYU  KUMAR  1.0
Kanhaiya KANHAIYA JEE   KANHAIYA  JEE  0.47058823529411764
KOTRA NIHARIKA   KOTRA  NIHARIKA  1.0
KOVADA SAKETHU   KOVADA  SAKETHU  1.0
RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  1.0
TRIPURNA NANDINI   TRIPURNA  NANDINI  1.0
AMIT KUMAR   AMIT  KU

P SRILAYA   P  SRILAYA  1.0
PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  1.0
POTRAKANTI SYAM   POTRAKANTI  SYAM  1.0
PRADHAN GANESH   PRADHAN  GANESH  1.0
RAJ ARYAN   RAJ  ARYAN  1.0
RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  1.0
SAKSHI KUMARI   SAKSHI  KUMARI  1.0
SANTOSH KUMAR SAHUKAR   SANTOSH  KUMAR SAHUKAR  1.0
SATISHKUMAR SINGH   SATISH KUMAR  SINGH  1.0
TRIPURNA NANDINI   TRIPURNA  NANDINI  1.0
VOONA NIDHI   VOONA SRI NIDHI  0.875
VYSYARAJU NIKHI   VYSYARAJU SRI NIKHI  0.8333333333333334
VYSYARAJU TARUN   VYSYARAJU  TARUN  1.0
GANGULA TARUN   GANGULA  TARUN  1.0
GONDU GANESH   GONDU  GANESH  1.0
KARUSODU KUMAR   KARUSODU  LAVANKUMAR  0.7272727272727273
MD IMRAN ANJUM   MDIMRAN  ANJUM  1.0
SATYAM SIDDHARTH   SATYAM  SIDDHARTH  1.0
Abhimanyu Kumar   ABHIMANYU  KUMAR  1.0
AMIT KUMAR   AMIT  KUMAR  1.0
ANUMANCHIPALLI ADITYA   ANUMANCHIPALLI VENKATA ADITYA  0.8421052631578947
ANURAG KUMAR   ANURAG  KUMAR  1.0
BABLU KUMAR   BABLU  KUMAR  1.0
BINODINI KAR   BINODINI  KAR  1.0
CHENNAKESAVULA 

PRADHAN GANESH   PRADHAN  GANESH  1.0
RAHUL KUMAR   RAHUL  KUMAR  1.0
RAJ ARYAN   RAJ  ARYAN  1.0
RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  1.0
RUPESH KUMAR   RUPESH  KUMAR  1.0
SAKSHI KUMARI   SAKSHI  KUMARI  1.0
SANDEEP KUMAR   SANDEEP  KUMAR  1.0
SATISHKUMAR SINGH   SATISH KUMAR  SINGH  1.0
Shashikant Raj   SHASHIKANT  RAJ  1.0
TRIPURNA NANDINI   TRIPURNA  NANDINI  1.0
VEMPATI SHANKAR   VEMPATI JAYA SHANKAR  0.8333333333333334
VOONA NIDHI   VOONA SRI NIDHI  0.875
VYSYARAJU NIKHI   VYSYARAJU SRI NIKHI  0.8333333333333334
VYSYARAJU TARUN   VYSYARAJU  TARUN  1.0
BINODINI KAR   BINODINI  KAR  1.0
JYOTI KUMARI   JYOTI  KUMARI  1.0
KARUSODU KUMAR   KARUSODU  LAVANKUMAR  0.7272727272727273
KOVADA SAKETHU   KOVADA  SAKETHU  1.0
POTRAKANTI SYAM   POTRAKANTI  SYAM  1.0
SANTOSH KUMAR SAHUKAR   SANTOSH  KUMAR SAHUKAR  1.0
P SRILAYA   P  SRILAYA  1.0
Abhimanyu Kumar   ABHIMANYU  KUMAR  1.0
AMIT KUMAR   AMIT  KUMAR  1.0
ANKIT KUMAR   ANKIT  KUMAR  1.0
ANUMANCHIPALLI ADITYA   ANUMANCHIPALLI VENK

CHIRANJEE V   CHIRANJEEV    1.0
GONDU GANESH   GONDU  GANESH  1.0
KARUSODU KUMAR   KARUSODU  LAVANKUMAR  0.7272727272727273
KOTRA NIHARIKA   KOTRA  NIHARIKA  1.0
MANISH KUMAR   MANISH  KUMAR  1.0
Niteshkumar Sah   NITESH KUMAR  SAH  1.0
P SRILAYA   P  SRILAYA  1.0
PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  1.0
PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  1.0
RAHUL KUMAR   RAHUL  KUMAR  1.0
RUPESH KUMAR   RUPESH  KUMAR  1.0
SANTOSH KUMAR SAHUKAR   SANTOSH  KUMAR SAHUKAR  1.0
SUNIL KUMAR   PEDIREDLA  KUSUMA  0.5
TRIPURNA NANDINI   TRIPURNA  NANDINI  1.0
VYSYARAJU NIKHI   VYSYARAJU SRI NIKHI  0.8333333333333334
VYSYARAJU TARUN   VYSYARAJU  TARUN  1.0
Gaurav KUMAR   GAURAV  KUMAR  1.0
Kanhaiya KANHAIYA JEE   KANHAIYA  JEE  0.47058823529411764
NIRAKAR BHOLO   NIRAKAR  BHOLO  1.0
RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  1.0
SAKSHI KUMARI   SAKSHI  KUMARI  1.0
SANDEEP KUMAR   SANDEEP  KUMAR  1.0
VEMPATI SHANKAR   VEMPATI JAYA SHANKAR  0.8333333333333334
ANKIT KUMAR   ANKIT  KUMAR  1.0
CHENNAKESAV

Niteshkumar Sah   NITESH KUMAR  SAH  1.0
PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  1.0
PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  1.0
PRADHAN GANESH   PRADHAN  GANESH  1.0
RAHUL KUMAR   RAHUL  KUMAR  1.0
RAJ ARYAN   RAJ  ARYAN  1.0
RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  1.0
SATYAM SIDDHARTH   SATYAM  SIDDHARTH  1.0
TRIPURNA NANDINI   TRIPURNA  NANDINI  1.0
VYSYARAJU TARUN   VYSYARAJU  TARUN  1.0
AMIT KUMAR   AMIT  KUMAR  1.0
BABLU KUMAR   BABLU  KUMAR  1.0
BINODINI KAR   BINODINI  KAR  1.0
Kanhaiya KANHAIYA JEE   KANHAIYA  JEE  0.47058823529411764
KARUSODU KUMAR   KARUSODU  LAVANKUMAR  0.7272727272727273
MANISH KUMAR   MANISH  KUMAR  1.0
MD SHAHID AKIL   MD SHAHID  AKIL  1.0
NAV RATNA BADAL   NAVRATNA  BADAL  1.0
RUPESH KUMAR   RUPESH  KUMAR  1.0
SAKSHI KUMARI   SAKSHI  KUMARI  1.0
SANDEEP KUMAR   SANDEEP  KUMAR  1.0
UTTAM KUMAR   UTTAM  KUMAR  1.0
VEMPATI SHANKAR   VEMPATI JAYA SHANKAR  0.8333333333333334
Abhimanyu Kumar   ABHIMANYU  KUMAR  1.0
ANUMANCHIPALLI ADITYA   ANUMANCHIPALLI 